In [1]:
import re
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchsummary import summary
from sentence_transformers import SentenceTransformer

from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

import faiss

In [2]:
####### read data
movies = pd.read_csv('Data2/movies.csv')
ratings = pd.read_csv('Data2/ratings.csv')
df = pd.merge(ratings, movies, on='movieId')
df.sort_values(by=['userId', 'timestamp'], inplace=True)

####### construct lookup tables for userId, movieId, and rating
userid2id = {val:idx for idx, val in enumerate(sorted(df.userId.unique()))}
movieId2id = {val:idx for idx, val in enumerate(sorted(df.movieId.unique()))}
rating2id = {val:idx for idx, val in enumerate(sorted(df.rating.unique()))}
 
df.loc[:, 'userId'] = df.userId.apply(lambda x: userid2id[x])
df.loc[:, 'movieId'] = df.movieId.apply(lambda x: movieId2id[x])
df.loc[:, 'rating'] = df.rating.apply(lambda x: rating2id[x])

####### construct datetime column and extract features from it
df.loc[:, 'timestamp'] = df.timestamp.apply(lambda x: datetime.fromtimestamp(x))

df['quarter'] = df.timestamp.dt.quarter
df['month'] = df.timestamp.dt.month
df['day'] = df.timestamp.dt.day
df['hour'] = df.timestamp.dt.hour
df['weekend'] = df.timestamp.apply(lambda x: 1 if x.dayofweek > 5 else 0)

## construct part_of_day columns with values [Morning, Afternoon, Evening, Night]
def get_part_of_day(datetime_obj):
    time_of_day = pd.to_datetime(datetime_obj).strftime('%H:%M:%S')
    part_of_day = ''
    
    if '06:00:00' <= time_of_day < '12:00:00':
        part_of_day = 'Morning'
    elif '12:00:00' <= time_of_day < '18:00:00':
        part_of_day = 'Afternoon'
    elif '18:00:00' <= time_of_day < '22:00:00':
        part_of_day = 'Evening'
    else:
        part_of_day = 'Night'
    
    return part_of_day

df['part_of_day'] = df.timestamp.apply(lambda x: get_part_of_day(x))

####### create decade dummy columns
def extract_decade(datetime_obj):
    year = datetime_obj.year
    start_year = (year // 10) * 10
    end_year = start_year + 9
    decade = f'{start_year}-{end_year}'
    return decade

df['release_decade'] = df.timestamp.apply(lambda x: extract_decade(x))

####### create movie_age columns which shows time after it's release
mx = df.timestamp.max()
df['movie_age'] = df.timestamp.apply(lambda x: (mx - x).days / 360)

####### create time variable which ordinally show watched movies
df['time'] = np.nan
for i in range(df.userId.unique().size):
    df.loc[df.userId==i, 'time'] = np.arange(1, df.loc[df.userId==i, 'time'].size+1, dtype=int)

df['time_sqrt'] = np.sqrt(df.time)
####### add genres dummy columns
genres = []

for idx, val in df[['genres']].itertuples():
    l = val.split('|')
    for genre in l:
        if genre not in genres:
            genres.append(genre)

for genre in genres:
    df[genre] = 0
    for idx, val in df[[genre]].itertuples():
        if genre in df.iloc[idx,5]:
            df.iloc[idx,-1] = 1

####### extract release year from title
def year_extractor(years_list):
    try:
        return years_list[0]
    except:
        return np.nan

df['release_year'] = df.title.apply(lambda x: re.findall(r"\b\d{4}\b", x)).apply(lambda x: year_extractor(x))
df['release_year'].fillna(1995, inplace=True)
df['release_year'] = df['release_year'].astype(int)

####### extract title name (without year part)
df['title_name'] = df.title.apply(lambda x: x[:x.rfind('(')])

####### reset index
df.reset_index(drop=True, inplace=True)

####### create time_dif column to express time passed after last watch
df['time_dif'] = np.nan
for i in range(df.userId.unique().shape[0]):
    new_df = df[df.userId==i][['timestamp']]
    lowest_idx, highest_idx = new_df.index[0], new_df.index[-1] + 1
    df.iloc[lowest_idx:highest_idx, -1] = new_df.timestamp.diff().dt.seconds
df.time_dif.fillna(0, inplace=True)
df['time_dif_sqrt'] = np.square(df['time_dif'])
df['time_dif_square'] = np.square(df['time_dif'])

####### convert time categorical columns to dummy columns
dummy_cols = ['quarter', 'month', 'day', 'part_of_day', 'weekend', 'release_decade']
df = pd.concat([df, pd.get_dummies(df[dummy_cols], columns=dummy_cols, drop_first=True)], axis=1)

####### assign embedding to each movie title (376 dimensional vector)
movie2embd = {}

model = SentenceTransformer('all-MiniLM-L6-v2')

for i in tqdm(list(df.movieId.unique())):
    movie2embd[i] = model.encode([list(df[df.movieId==i].title_name)[0].strip()])

####### assign each title to the 376 dimensional vector
df['watched_movies'] = df.index.map(lambda x: np.array(x))

####### select new df
new_df = df[['userId','movieId', 'rating', 'time']]
new_df.head()

####### construct variable which shows last 50 watched movies for each user in a particular time
ls = {}
for i in tqdm(range(df.userId.unique().shape[0])): 
    new_df = df[df.userId==i][['userId','movieId', 'rating', 'time']]
    new_df.reset_index(inplace=True)
    for j in range(new_df.shape[0]):
        features = new_df.iloc[j, :]
        if features.name == 0:
            ls[features.name] = [10325]
        else:
            idx = features.name
            new_df.iloc[idx-1,1]
            ls[features['index']] = list(new_df.iloc[:idx,2])

for i in range(105339):
    try:
        ls[i]
    except:
        ls[i] = [10325]

def func(x):
    return x[-50:]

ls = {k:func(v) for k, v in ls.items()}

def padder(x):
    return x + (50 - len(x)) * [10325]

ls = {k:padder(v) for k, v in ls.items()}

df['watched_movies'] = df.index.map(lambda x: np.array(ls[x]))

###### add 3 categorical class for rating
# df['like'], df['neutral'], df['dislike'] = (df.rating >=8).astype(int), ((df.rating >=6)==(df.rating <8)).astype(int), (df.rating<6).astype(int)
def rating_mapper(rating):
    if rating >=7:
        return 0
    elif (rating < 7) and (rating > 5):
        return 1
    else:
        return 2

df['user_reaction'] = df.rating.apply(lambda x: rating_mapper(x))

###### add average current for each observation
df['avg_current_rating'] = np.nan

for movie_idx in range(df.movieId.unique().shape[0]):
    df1 = df[df.movieId==movie_idx].sort_values(by=['movieId', 'timestamp'])[['movieId', 'timestamp', 'rating']]
    for i in range(df1.shape[0]):
        idx, (movieid, timest, _) = df1.iloc[i,:].name, df1.iloc[i,:]
        df.iloc[idx,-1] = df1[df1.timestamp<timest]['rating'].mean()

df['has_0_review'] = df.avg_current_rating.apply(lambda x: 1 if np.isnan(x) else 0)
df.avg_current_rating.fillna(0, inplace=True)

df['avg_current_rating_sqrt'] = np.sqrt(df['avg_current_rating'])
df['avg_current_rating_suare'] = np.square(df['avg_current_rating'])

/tmp/ipykernel_4048/2797731845.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'rating'] = df.rating.apply(lambda x: rating2id[x])
100%|██████████| 668/668 [00:14<00:00, 45.02it/s]


In [115]:
new_df = df[['userId','movieId', 'rating', 'time']]
new_df.head()

####### construct variable which shows last 50 watched movies for each user in a particular time
ls = {}
for i in tqdm(range(df.userId.unique().shape[0])): 
    new_df = df[df.userId==i][['userId','movieId', 'rating', 'time']]
    new_df.reset_index(inplace=True)
    new_df.rating = new_df.rating + 1
    for j in range(new_df.shape[0]):
        features = new_df.iloc[j, :]
        if features.name == 0:
            ls[features.name] = [0.01]
        else:
            idx = features.name
            new_df.iloc[idx-1,1]
            ls[features['index']] = list(new_df.iloc[:idx,3])

for i in range(105339):
    try:
        ls[i]
    except:
        ls[i] = [0.01]

def func(x):
    return x[-50:]

ls = {k:func(v) for k, v in ls.items()}

def padder(x):
    return x + (50 - len(x)) * [0.01]

ls = {k:padder(v) for k, v in ls.items()}

# df['watched_movies'] = df.index.map(lambda x: np.array(ls[x]))

100%|██████████| 668/668 [00:11<00:00, 57.62it/s]


In [122]:
# len(ls[0]), len(ls[1]), len(ls[2])
# ls[3]

In [86]:
for user_idx in range(df.userId.unique().shape[0]):
    new_df = df[df.userId==user_idx]
    break

In [125]:
user_idx

0

In [103]:
for _, userId, movieId, rating, watched_movies in df[['userId', 'movieId', 'rating', 'watched_movies']].itertuples():
    print(userId, movieId, rating)
    print(list(set(watched_movies) - set([10325])))
    break

0 2712 8
[]


In [132]:
k = list(set(watched_indices) & set(reccomended_movies_indices.numpy()))
k

[1664, 226, 1026, 231, 1962, 1716, 2357, 246, 2712, 1916]

In [127]:
watched_indices, reccomended_movies_indices

(array([ 2712,  1916,  2357,   231,  1962,   226,  1026,   246,  1716,
         1664, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
        10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
        10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
        10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
        10325, 10325, 10325, 10325, 10325]),
 tensor([ 176, 1664,  613,  246, 1583, 1716,  610, 2596, 1026,  279,  863,  226,
          230, 1962,  231, 2357, 1684, 3885, 1916,  230, 3379, 2056, 2098, 1640,
          426, 2712, 2441,  525, 2032, 1126,  971, 1583,  848,  126,  236, 2974,
          613,  246,  861,  231, 1126, 1590, 7621,  969, 4172,  995, 1696, 1360,
         1367, 1295,  426, 2162,  986, 2245,  993, 3637,  956, 2402,  890,  188,
          995, 2197, 2172, 9365, 1716, 1706, 1093, 1481, 1728,  279, 1861, 2168,
         1143, 2769,  894,  525,  518, 7216, 2763,  445,  646, 1256, 1029,  646,
         2056, 3555, 1659, 2056, 

In [75]:
watched_indices = val_df.iloc[10,:].watched_movies      # watched videos
reccomended_movies_indices = val_vecs[10]               # reccomended videos

In [84]:
reccomended_movies_indices

tensor([ 176, 1664,  613,  246, 1583, 1716,  610, 2596, 1026,  279,  863,  226,
         230, 1962,  231, 2357, 1684, 3885, 1916,  230, 3379, 2056, 2098, 1640,
         426, 2712, 2441,  525, 2032, 1126,  971, 1583,  848,  126,  236, 2974,
         613,  246,  861,  231, 1126, 1590, 7621,  969, 4172,  995, 1696, 1360,
        1367, 1295,  426, 2162,  986, 2245,  993, 3637,  956, 2402,  890,  188,
         995, 2197, 2172, 9365, 1716, 1706, 1093, 1481, 1728,  279, 1861, 2168,
        1143, 2769,  894,  525,  518, 7216, 2763,  445,  646, 1256, 1029,  646,
        2056, 3555, 1659, 2056,  126,  279, 2238,  956,  126, 2145, 1060, 2030,
         471, 2712,  279,   98])

In [83]:
watched_indices

array([ 2712,  1916,  2357,   231,  1962,   226,  1026,   246,  1716,
        1664, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
       10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
       10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
       10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325, 10325,
       10325, 10325, 10325, 10325, 10325])

In [82]:
denominator = len(set(reccomended_movies_indices.numpy()) - set(watched_indices))
denominator

70

In [6]:
dummy_cols = ['weekend', 'hour', 'quarter_2', 'quarter_3', 'quarter_4', 'month_2', 'month_3', 'month_4',
              'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
              'month_11', 'month_12', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6',
              'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13',
              'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20',
              'day_21', 'day_22', 'day_23', 'day_24', 'day_25', 'day_26', 'day_27',
              'day_28', 'day_29', 'day_30', 'day_31', 'part_of_day_Evening',
              'part_of_day_Morning', 'part_of_day_Night', 'weekend_1']

####### select train_val_test splits
train_df = df[df.movieId>80]
val_df = df[df.userId<=40]
test_df = df[(df.userId>40)&(df.userId<=80)]

X_int_train = train_df['movieId'].values                                                 # indices of target movies
Xd_train = train_df[dummy_cols].values                                                   # user/time features
Xw_train = np.array([np.array(i,dtype=int) for i in train_df['watched_movies'].values])  # indices of last 50 movies watched

X_int_val = val_df[ 'movieId'].values
Xd_val = val_df[dummy_cols].values
Xw_val = np.array([np.array(i,dtype=int) for i in val_df['watched_movies'].values]) 

X_int_test = test_df[ 'movieId'].values
Xd_test = test_df[dummy_cols].values
Xw_test = np.array([np.array(i,dtype=int) for i in test_df['watched_movies'].values]) 

X_int_train, Xd_train = torch.Tensor(X_int_train).long(), torch.Tensor(Xd_train).float()
Xw_train = torch.Tensor(Xw_train).long()

X_int_val, Xd_val = torch.Tensor(X_int_val).long(), torch.Tensor(Xd_val).float()
Xw_val = torch.Tensor(Xw_val).long()

X_int_test, Xd_test = torch.Tensor(X_int_test).long(), torch.Tensor(Xd_test).float()
Xw_test = torch.Tensor(Xw_test).long()

In [7]:
columns_for_ranking = ['release_year', 'time_dif', 'time_dif_sqrt', 'time_dif_square', 'avg_current_rating',
                       'has_0_review', 'avg_current_rating_sqrt', 'avg_current_rating_suare', 'time', 'time_sqrt']


In [10]:
class DatasetForCandidateGenerator(Dataset):
    def __init__(self, X_int, Xd, Xw):
        self.X_int = X_int
        self.Xd = Xd
        self.Xw = Xw
    
    def __len__(self):
        return self.X_int.shape[0]
    
    def __getitem__(self, idx):
        return self.X_int[idx], self.Xd[idx], self.Xw[idx]

# class DatasetForRanking(Dataset):
#     def __init__(self, )

train_dataset = DatasetForCandidateGenerator(X_int_train, Xd_train, Xw_train)
val_dataset = DatasetForCandidateGenerator(X_int_val, Xd_val, Xw_val)
test_dataset = DatasetForCandidateGenerator(X_int_test, Xd_test, Xw_test)

train_loader = DataLoader(train_dataset, batch_size=5000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=3000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=3000, shuffle=False)

In [11]:
a, b, c = next(iter(train_loader))

In [12]:
class CandidatesGenerator(nn.Module):
    def __init__(self, 
                 n_movies,
                 movie_emb_dim,    
                 sparse_matrix_dim,
                 hidden_dim):
        super().__init__()
        # embedding for user & movies
        self.movie_embd = nn.Embedding(n_movies+1, movie_emb_dim, padding_idx=n_movies)

        # linear layers
        self.ln1 = nn.Linear(movie_emb_dim + sparse_matrix_dim, hidden_dim * 4)
        self.ln2 = nn.Linear(hidden_dim * 4, hidden_dim * 2)

        self.classifier = nn.Linear(hidden_dim * 2, n_movies)

        # dropout
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, Xd, Xw):
        watched_embedding = torch.mean(self.movie_embd(Xw.long()), axis=1) # (batch_size, movie_emb_dim)
        out = torch.cat([Xd, watched_embedding], axis=1)

        out = self.ln1(out)
        out = F.leaky_relu(out, negative_slope=0.2)
        out = self.dropout(out)

        out = self.ln2(out)
        logits = F.leaky_relu(out, negative_slope=0.2)
        logits = self.dropout(logits)

        logits = self.classifier(logits)
        return out, logits

In [13]:
model = CandidatesGenerator(n_movies=10325,
                            movie_emb_dim=32,    
                            sparse_matrix_dim=50,
                            hidden_dim=64)

In [14]:
embds, logits = model(b,c)
embds.shape, logits.shape

(torch.Size([5000, 128]), torch.Size([5000, 10325]))

In [15]:
lr_rate = 0.01
n_epoch = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=False)

In [291]:
train_epoch_losses, val_epoch_losses = [], []

for epoch in range(n_epoch):
    train_epoch_loss, val_epoch_loss = 0, 0
    train_epoch_acc, val_epoch_acc = 0, 0
    
    model.train()
    for n_batch, (X_int, Xd, Xw) in enumerate(train_loader):
        y_hat, logits = model(Xd, Xw)
        loss = loss_fn(logits, X_int)
        
        optimizer.zero_grad()
        loss.backward()
    
        optimizer.step()
        train_epoch_loss += loss.item()
        train_epoch_acc += (torch.argmax(F.softmax(logits, dim=1), axis=1) == X_int).sum().item() /  X_int.size()[0]
    
    model.eval()
    with torch.no_grad():
        for n_batch, (X_int, Xd, Xw) in enumerate(val_loader):
            y_hat, logits = model(Xd, Xw)
            loss = loss_fn(logits, X_int)
            val_epoch_loss += loss.item()
            val_epoch_acc += (torch.argmax(F.softmax(logits, dim=1), axis=1) == X_int).sum().item() /  X_int.size()[0]
    
    train_epoch_losses.append(train_epoch_loss/len(train_loader))
    val_epoch_losses.append(val_epoch_loss/len(val_loader))
    scheduler.step(train_epoch_loss/len(train_loader))
    print(f'Epoch: {epoch+1} | Train Loss: {train_epoch_loss/len(train_loader):.3} | Train Acc: {train_epoch_acc / len(train_loader):.3} \
| Val Loss: {val_epoch_loss/len(val_loader):.3} | Vall Acc: {val_epoch_acc / len(val_loader):.3}')

Epoch: 1 | Train Loss: 5.38 | Train Acc: 0.0705 | Val Loss: 5.52 | Vall Acc: 0.0855
Epoch: 2 | Train Loss: 5.36 | Train Acc: 0.0733 | Val Loss: 5.48 | Vall Acc: 0.0896
Epoch: 3 | Train Loss: 5.33 | Train Acc: 0.0744 | Val Loss: 5.43 | Vall Acc: 0.092
Epoch: 4 | Train Loss: 5.3 | Train Acc: 0.0762 | Val Loss: 5.4 | Vall Acc: 0.0903
Epoch: 5 | Train Loss: 5.28 | Train Acc: 0.0773 | Val Loss: 5.39 | Vall Acc: 0.0971
Epoch: 6 | Train Loss: 5.26 | Train Acc: 0.0796 | Val Loss: 5.37 | Vall Acc: 0.0981
Epoch: 7 | Train Loss: 5.22 | Train Acc: 0.0821 | Val Loss: 5.33 | Vall Acc: 0.0983
Epoch: 8 | Train Loss: 5.22 | Train Acc: 0.0811 | Val Loss: 5.33 | Vall Acc: 0.0955
Epoch: 9 | Train Loss: 5.19 | Train Acc: 0.0839 | Val Loss: 5.3 | Vall Acc: 0.103
Epoch: 10 | Train Loss: 5.16 | Train Acc: 0.086 | Val Loss: 5.29 | Vall Acc: 0.108


In [19]:
model.load_state_dict(torch.load('weights/weights1.pt'))

<All keys matched successfully>

In [20]:
train_embds = None
train_ints = None
model.eval()
with torch.no_grad():
    for n_batch, (X_int, Xd, Xw) in enumerate(train_loader):
        y_hat, _ = model(Xd, Xw)
        if train_embds == None:
            train_embds = y_hat
            train_ints = X_int
        else:
            train_embds = torch.cat([train_embds, y_hat], axis=0)
            train_ints = torch.cat([train_ints, X_int], axis=0)

In [21]:
val_embds = None
val_ints = None
model.eval()
with torch.no_grad():
    for n_batch, (X_int, Xd, Xw) in enumerate(val_loader):
        y_hat, _ = model(Xd, Xw)
        if val_embds == None:
            val_embds = y_hat
            val_ints = X_int
        else:
            val_embds = torch.cat([val_embds,y_hat], axis=0)
            val_ints = torch.cat([val_ints,X_int], axis=0)

In [22]:
test_embds = None
test_ints = None
model.eval()
with torch.no_grad():
    for n_batch, (X_int, Xd, Xw) in enumerate(test_loader):
        y_hat, _ = model(Xd, Xw)
        if test_embds == None:
            test_embds = y_hat
            test_ints = X_int
        else:
            test_embds = torch.cat([test_embds,y_hat], axis=0)
            test_ints = torch.cat([test_ints,X_int], axis=0)

In [55]:
train_embds.shape, val_embds.shape, test_embds.shape

(torch.Size([101947, 128]), torch.Size([3944, 128]), torch.Size([5388, 128]))

In [57]:
index = faiss.IndexFlatL2(128)
index.add(train_embds.detach().numpy())

val_scores, val_indices = index.search(val_embds.detach().numpy(), 100)
test_scores, test_indices = index.search(test_embds.detach().numpy(), 100)

In [24]:
val_vecs = train_ints[val_indices]
n1 = 0
for i in range(val_ints.shape[0]):
    if val_ints[i] in val_vecs[i]:
        n1+=1
test_vecs = train_ints[test_indices]
n2 = 0
for i in range(test_ints.shape[0]):
    if test_ints[i] in test_vecs[i]:
        n2+=1

vall_acc = round(n1/val_ints.shape[0], 2)
test_acc = round(n2/test_ints.shape[0], 2)
print(f'Validation Set Accracy: {vall_acc}')
print(f'Test Set Accuracy: {test_acc}')

Validation Set Accracy: 0.95
Test Set Accuracy: 0.97


In [28]:
val_vecs[0]

tensor([  426,  2712,  1640,  1684,  1916,  2357,   231,  1962,   230,   236,
         2098,   226,  1026,  1590,   246,   956,  1367,  1481,  2441,  7216,
         1126,   230,   445,   894,  2245,  1728,   969,  6844,   995,   613,
          279,  2056,   986,  1360,  1126,  1716,  1143,  1664,   176,  1843,
          958,   231,  4490,   126,  1029,  2056,  3885,  2056,  2974,  5206,
          471,   246,   644,  6412,   279,  1295,   188, 10243,  3885,   646,
          958,   230,  1367,  9365,  2162,  2056,  3379,  2596,  3562,   126,
          426,  1367,  2763,   316,  1583,  1861,   874,   313,   960,   471,
          525,   993,   525,   690,   610,  2056,   426,  1214,   695,   126,
          279,  3555,  9996,   390,  1929,  2168,   960,   525,   426,  1070])

In [639]:
df.rating.unique()

array([8, 4, 6, 2, 3, 0, 7, 9, 5, 1])

In [649]:
val_embds[0].shape

torch.Size([128])

In [453]:
class ReccomenderSystem(nn.Module):
    def __init__(self, 
                 movie_emb_dim,    
                 title_embd_dim=376,
                 hidden_dim=128):
        super().__init__()

        self.ln1 = 

SyntaxError: invalid syntax (1102136943.py, line 1)